In [1]:
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import pandas as pd
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

In [2]:
dic = {
    "Nazare-Surf-Report":194,
    "Pipeline-Backdoor-Surf-Report":616,
    "Mundaka-Surf-Report": 169,
    "El-Fronton-Surf-Report":1879,
    "The-Box-Surf-Report":2873,
    "Puerto-Escondido-Surf-Report":340,
    "Itacoatiara-Surf-Report":1273,
    "Padang-Padang-Surf-Report":1121,
    "Mosca-Point-Surf-Report":1875,
    "Tauro-Surf-Report":1878
       
}

In [56]:
range_2019 = [("2019-01-01","2019-01-31"),("2019-02-01","2019-02-28"),("2019-03-01","2019-03-31"),("2019-04-01","2019-04-30"),("2019-05-01","2019-05-31"),("2019-06-01","2019-06-30"),("2019-07-01","2019-07-31"),("2019-08-01","2019-08-31"),("2019-09-01","2019-09-30"),("2019-10-01","2019-10-31"),("2019-11-01","2019-11-30"),("2019-12-01","2019-12-31")]

In [179]:
ranges_2018 = [("2018-01-01","2018-01-31"),("2018-02-01","2018-02-28"),("2018-03-01","2018-03-31"),("2018-04-01","2018-04-30"),("2018-05-01","2018-05-31"),("2018-06-01","2018-06-30"),("2018-07-01","2018-07-31"),("2018-08-01","2018-08-31"),("2018-09-01","2018-09-30"),("2018-10-01","2018-10-31"),("2018-11-01","2018-11-30"),("2018-12-01","2018-12-31")]

In [284]:
def login():
    driver = webdriver.Chrome("./chromedriver.exe")
    driver.get("https://magicseaweed.com/account/login/?dest=%2F")
    sleep(2)
    driver.find_element_by_css_selector("#msw-js-user-login-tab > form > div:nth-child(2) > input").send_keys("lukesmith194@hotmail.com")
    driver.find_element_by_css_selector("#msw-js-user-login-tab > form > div:nth-child(3) > input").send_keys("Versus.123")
    sleep(1)
    driver.find_element_by_css_selector("#msw-js-login").click()
    sleep(2)

    return driver

In [285]:
driver = login()

In [286]:
def get_arrow_value(fila):
    col = fila.find_element_by_class_name("text-center.msw-js-tooltip.background-gray-lighter").get_attribute("data-original-title")
    direction, angle = col.split(" - ")
    
    angle = int(angle[:-1])
    
    return direction, angle

In [287]:
def get_wind_value(fila):
    try:
        col = fila.find_element_by_class_name("text-center.last.msw-js-tooltip.td-square.background-success").get_attribute("data-original-title")
    except:
        try:
            col = fila.find_element_by_class_name("text-center.last.msw-js-tooltip.td-square.background-warning").get_attribute("data-original-title")
        except:
            try:
                col = fila.find_element_by_class_name("text-center.last.msw-js-tooltip.td-square.background-danger").get_attribute("data-original-title")
            except:
                return "null", "null", np.nan
        
    
    text, angle = col.split(" - ")

    angle = int(angle[:-1])
    _, text = text.split(", ")

    text, direction = text.split()

    return text, direction, angle

In [288]:
def get_hora(fila):
    elems = fila.text.split("\n")
    elems = elems[0].split() + elems[1:]
    
    arrow = get_arrow_value(fila)
    wind = get_wind_value(fila)
    
    elems = elems + list(arrow) + list(wind)
    
    return elems

In [289]:
def get_dia(tabla):
    filas = tabla.find_elements_by_tag_name("tr")
    dia = filas[0].text.split()[1]
    
    # 6am 12pm 6pm son filas 4, 6, 8
    filas = filas[3:8:2]
    
    dias = [[dia] + get_hora(f) for f in filas]
    
    return dias

In [295]:
def get_range_dates(d, driver):
    # d is a tuple of 2 elements (start_date, end_date)
    # access webpage
        
    driver.get(f"https://magicseaweed.com/Mosca-Point-Surf-Report/1875/Historic/?start={d[0]}&end={d[1]}")
    sleep(2)
    # extract 31 days' data
    table = driver.find_element_by_css_selector("#msw-js-fc > div.table-responsive-xs > table")

    tablitas = table.find_elements_by_tag_name("tbody")
    tablitas = tablitas[::3]

    mega_tabla = []

    for t in tablitas:
        dia = get_dia(t)
        mega_tabla.extend(dia)
        
    return mega_tabla

In [296]:
tabla_anio = []

for r in ranges_2018:
    print(f"Rango {r}...")
    driver.get(f"https://magicseaweed.com/Mosca-Point-Surf-Report/1875/Historic/?start={r[0]}&end={r[1]}")
    
    tabla_mes = get_range_dates(r, driver)
    
    tabla_anio.extend(tabla_mes)

Rango ('2018-01-01', '2018-01-31')...
Rango ('2018-02-01', '2018-02-28')...
Rango ('2018-03-01', '2018-03-31')...
Rango ('2018-04-01', '2018-04-30')...
Rango ('2018-05-01', '2018-05-31')...
Rango ('2018-06-01', '2018-06-30')...
Rango ('2018-07-01', '2018-07-31')...
Rango ('2018-08-01', '2018-08-31')...
Rango ('2018-09-01', '2018-09-30')...
Rango ('2018-10-01', '2018-10-31')...
Rango ('2018-11-01', '2018-11-30')...
Rango ('2018-12-01', '2018-12-31')...


In [297]:
df_mosca_2018 = pd.DataFrame(tabla_anio, columns=[
    "date", "time", "range", "height", "period", "wind_mph", "wind_kmh", "mph", "temp", "prob",
    "swell_dir", "swell_angle", "wind", "wind_dir", "wind_angle",#"x1","x2","x3","x4"
])

## 2019

### Fronton

In [178]:
df_fronton_2019

,date,time,range,height,period,wind_mph,wind_kmh,mph,temp,prob,swell_dir,swell_angle,wind,wind_dir,wind_angle,prueba
0,01/01/2019,6am,2-3ft,2ft,14s,10,12,mph,14°c,100%,WNW,299,Offshore,SE,135,2019-01-01
1,01/01/2019,12pm,1-2ft,2ft,14s,12,14,mph,20°c,100%,WNW,301,Offshore,SE,137,2019-01-01
2,01/01/2019,6pm,1-2ft,1.8ft,13s,8,10,mph,17°c,100%,WNW,302,Cross/Offshore,E,100,2019-01-01
3,02/01/2019,6am,1-2ft,1.5ft,12s,8,10,mph,14°c,100%,NW,308,Cross/Offshore,ESE,102,2019-02-01
4,02/01/2019,12pm,1-2ft,1.4ft,11s,10,11,mph,19°c,100%,NW,310,Cross/Offshore,ESE,104,2019-02-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,30/12/2019,12pm,1-2ft,1.5ft,13s,9,11,mph,19°c,100%,NW,309,Offshore,ESE,121,2019-12-30
1091,30/12/2019,6pm,2-3ft,1.4ft,16s,4,5,mph,17°c,100%,NW,326,Cross/Offshore,ESE,104,2019-12-30
1092,31/12/2019,6am,2-3ft,1.9ft,14s,5,7,mph,13°c,100%,NW,319,Cross/Onshore,ENE,72,2019-12-31
1093,31/12/2019,12pm,2-3ft,1.8ft,13s,10,10,mph,18°c,100%,NW,320,Offshore,SE,124,2019-12-31


In [44]:
df_fronton_2019.to_csv("df_fronton_2019.csv", index=False)

### Pipe

In [54]:
df_pipe_2019.head()

,date,time,range,height,period,wind_mph,wind_kmh,mph,temp,prob,swell_dir,swell_angle,wind,wind_dir,wind_angle
0,01/01,6am,3-5ft,2.5ft,16s,14,24,mph,21°c,100%,NW,316,Offshore,E,93
1,01/01,12pm,4-6ft,4ft,16s,14,20,mph,24°c,100%,NW,315,Offshore,E,86
2,01/01,6pm,5-8ft,5.5ft,16s,8,17,mph,20°c,100%,NW,314,Cross/Offshore,E,82
3,02/01,6am,6-9ft,6ft,15s,12,22,mph,20°c,100%,NW,313,Cross/Offshore,E,81
4,02/01,12pm,6-9ft,6ft,15s,14,20,mph,24°c,100%,NW,313,Cross/Offshore,E,81


In [55]:
df_pipe_2019.to_csv("df_pipe_2019.csv", index=False)

### Mundaka

In [68]:
df_mundaka_2019.head()

,date,time,range,height,period,wind_mph,wind_kmh,mph,temp,prob,swell_dir,swell_angle,wind,wind_dir,wind_angle
0,01/01,6am,1ft,1ft,5s,3,3,mph,9°c,100%,N,7,Onshore,ENE,72
1,01/01,12pm,1ft,0.9ft,10s,4,3,mph,10°c,100%,NW,318,Onshore,NNE,14
2,01/01,6pm,1ft,1ft,4s,5,5,mph,9°c,100%,N,9,Onshore,NE,36
3,02/01,6am,1ft,0.7ft,13s,9,9,mph,8°c,100%,NW,312,Onshore,E,81
4,02/01,12pm,1ft,1.1ft,12s,10,12,mph,9°c,100%,WNW,301,Onshore,E,83


In [69]:
df_mundaka_2019.to_csv("df_mundaka_2019.csv", index=False)

### Nazare

In [79]:
df_nazare_2019

,date,time,range,height,period,wind_mph,wind_kmh,mph,temp,prob,swell_dir,swell_angle,wind,wind_dir,wind_angle
0,01/01,6am,6-9ft,3.5ft,14s,3,4,mph,7°c,100%,W,267,Offshore,ESE,103
1,01/01,12pm,6-9ft,3ft,13s,1,2,mph,15°c,100%,W,270,Offshore,E,81
2,01/01,6pm,5-8ft,3ft,12s,4,5,mph,9°c,100%,W,275,Cross/Onshore,N,2
3,02/01,6am,4-7ft,2.5ft,11s,5,6,mph,7°c,100%,W,274,Cross/Onshore,N,10
4,02/01,12pm,3-5ft,2ft,10s,6,10,mph,14°c,100%,WSW,254,Onshore,NNW,343
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,30/12,12pm,6-9ft,3.5ft,12s,2,2,mph,15°c,100%,WNW,287,Cross/Offshore,S,183
1091,30/12,6pm,5-8ft,3ft,12s,2,3,mph,9°c,100%,WNW,290,Cross/Offshore,NNE,31
1092,31/12,6am,5-8ft,3ft,11s,3,4,mph,6°c,100%,WNW,287,Cross/Offshore,NE,35
1093,31/12,12pm,5-8ft,3ft,11s,4,7,mph,13°c,100%,WNW,289,Cross/Onshore,N,6


In [80]:
df_nazare_2019.to_csv("df_nazare_2019.csv", index=False)

###  Box

In [99]:
df_box_2019.head()

,date,time,range,height,period,wind_mph,wind_kmh,mph,temp,prob,swell_dir,swell_angle,wind,wind_dir,wind_angle
0,01/01,6am,4-7ft,6.5ft,13s,7,10,mph,20°c,100%,SW,223,Offshore,SE,137
1,01/01,12pm,5-7ft,7ft,14s,13,15,mph,25°c,100%,SW,222,Onshore,SSW,205
2,01/01,6pm,6-10ft,9ft,16s,13,20,mph,20°c,100%,SW,225,Cross/Offshore,SSE,168
3,02/01,6am,5-8ft,8ft,14s,10,12,mph,19°c,100%,SW,225,Onshore,SW,234
4,02/01,12pm,5-8ft,7.5ft,13s,13,16,mph,21°c,100%,SW,227,Onshore,W,274


In [100]:
df_box_2019.to_csv("df_box_2019.csv", index=False)

### Puerto Escondido


In [119]:
df_pe_2019.head()

,date,time,range,height,period,wind_mph,wind_kmh,mph,temp,prob,swell_dir,swell_angle,wind,wind_dir,wind_angle
0,01/01,6am,3-5ft,1.9ft,12s,6,6,mph,20°c,100%,S,190,Offshore,NNE,14
1,01/01,12pm,3-5ft,2ft,12s,6,5,mph,31°c,100%,S,188,Onshore,SSW,210
2,01/01,6pm,3-5ft,2ft,12s,2,2,mph,22°c,100%,S,188,Onshore,SW,232
3,02/01,6am,3-4ft,1.7ft,11s,5,6,mph,21°c,100%,S,187,Offshore,NNE,15
4,02/01,12pm,2-4ft,1.7ft,11s,5,4,mph,32°c,100%,S,186,Onshore,S,191


In [120]:
df_pe_2019.to_csv("df_pe_2019.csv", index=False)

###  Itacoatiara

In [131]:
df_ita_2019.head()

,date,time,range,height,period,wind_mph,wind_kmh,mph,temp,prob,swell_dir,swell_angle,wind,wind_dir,wind_angle
0,01/01,6am,1ft,1.1ft,5s,4,5,mph,26°c,100%,SSE,151,Offshore,WNW,300
1,01/01,12pm,1-2ft,1ft,9s,13,12,mph,29°c,100%,S,191,Onshore,SSE,157
2,01/01,6pm,1ft,1.2ft,5s,8,10,mph,28°c,100%,SSE,160,Cross/Onshore,ESE,112
3,02/01,6am,1ft,1ft,8s,1,2,mph,26°c,100%,S,190,Cross-shore,W,279
4,02/01,12pm,1ft,0.8ft,8s,10,11,mph,30°c,100%,S,188,Onshore,SSE,153


In [132]:
df_ita_2019.to_csv("df_ita_2019.csv", index=False)

### Padang-Padang

In [142]:
df_pp_2019.head()

,date,time,range,height,period,wind_mph,wind_kmh,mph,temp,prob,swell_dir,swell_angle,wind,wind_dir,wind_angle
0,01/01,6am,Plano,4.5ft,12s,14,17,mph,28°c,100%,SSW,204,Cross/Onshore,WNW,291
1,01/01,12pm,Plano,4.5ft,12s,19,21,mph,29°c,100%,SSW,205,Onshore,W,268
2,01/01,6pm,Plano,7.5ft,8s,25,30,mph,28°c,100%,W,259,Onshore,W,271
3,02/01,6am,Plano,6ft,8s,19,22,mph,28°c,100%,WSW,258,Onshore,W,281
4,02/01,12pm,Plano,6ft,7s,24,28,mph,28°c,100%,W,259,Onshore,W,268


In [143]:
df_pp_2019.to_csv("df_pp_2019.csv", index=False)

### Mosca

In [165]:
df_mosca_2019.head()

,date,time,range,height,period,wind_mph,wind_kmh,mph,temp,prob,swell_dir,swell_angle,wind,wind_dir,wind_angle
0,01/01,6am,1-2ft,1.7ft,5s,10,12,mph,14°c,100%,E,99,Onshore,SE,135
1,01/01,12pm,2-3ft,3ft,6s,12,14,mph,20°c,100%,ESE,107,Onshore,SE,137
2,01/01,6pm,2-3ft,3ft,6s,8,10,mph,17°c,100%,ESE,111,Onshore,E,100
3,02/01,6am,1-2ft,2ft,5s,8,10,mph,14°c,100%,E,100,Onshore,ESE,102
4,02/01,12pm,1-2ft,2ft,5s,10,11,mph,19°c,100%,E,98,Onshore,ESE,104


In [155]:
df_mosca_2019.to_csv("df_mosca_2019.csv", index=False)

### Tauro

In [166]:
df_tauro_2019.head()

,date,time,range,height,period,wind_mph,wind_kmh,mph,temp,prob,swell_dir,swell_angle,wind,wind_dir,wind_angle
0,01/01,6am,2-3ft,2.5ft,14s,10,12,mph,14°c,100%,WNW,286,Cross-shore,SE,135
1,01/01,12pm,2-3ft,2ft,13s,12,14,mph,20°c,100%,WNW,286,Cross-shore,SE,137
2,01/01,6pm,1-2ft,2ft,13s,8,10,mph,17°c,100%,WNW,286,Offshore,E,100
3,02/01,6am,1-2ft,1.5ft,12s,8,10,mph,14°c,100%,WNW,286,Offshore,ESE,102
4,02/01,12pm,1-2ft,1.3ft,11s,10,11,mph,19°c,100%,WNW,287,Offshore,ESE,104


In [167]:
df_tauro_2019.to_csv("df_tauro_2019.csv", index=False)

## 2018

### Tauro

In [189]:
df_tauro_2018.head()

,date,time,range,height,period,wind_mph,wind_kmh,mph,temp,prob,swell_dir,swell_angle,wind,wind_dir,wind_angle
0,01/01,6am,0-1ft,1.1ft,14s,12,19,mph,15°c,100%,NW,314,Offshore,NE,42
1,01/01,12pm,1ft,1.1ft,13s,13,19,mph,18°c,100%,NW,311,Offshore,NE,45
2,01/01,6pm,0-1ft,1ft,13s,14,21,mph,17°c,100%,NW,308,Offshore,NNE,31
3,02/01,6am,Plano,1.7ft,14s,11,16,mph,15°c,100%,NW,324,Offshore,NE,44
4,02/01,12pm,1ft,0.7ft,14s,13,19,mph,20°c,100%,WNW,297,Offshore,ENE,60


In [190]:
df_tauro_2018.to_csv("df_tauro_2018.csv", index=False)

### Nazare

In [198]:
df_nazare_2018.head()

,date,time,range,height,period,wind_mph,wind_kmh,mph,temp,prob,swell_dir,swell_angle,wind,wind_dir,wind_angle
0,01/01,6am,20-30ft,15.5ft,15s,5,6,mph,8°c,100%,WNW,292,Cross/Offshore,S,186
1,01/01,12pm,18-28ft,14.5ft,15s,10,14,mph,15°c,100%,WNW,292,Onshore,W,277
2,01/01,6pm,18-26ft,13.5ft,14s,4,6,mph,11°c,100%,WNW,293,Onshore,WSW,253
3,02/01,6am,16-26ft,12.5ft,14s,4,5,mph,10°c,100%,WNW,297,Cross-shore,SSW,207
4,02/01,12pm,14-24ft,11.5ft,14s,7,11,mph,16°c,100%,WNW,297,Onshore,WSW,258


In [199]:
df_nazare_2018.to_csv("df_nazare_2018.csv", index=False)

### Pipe

In [212]:
df_pipe_2018.head()

,date,time,range,height,period,wind_mph,wind_kmh,mph,temp,prob,swell_dir,swell_angle,wind,wind_dir,wind_angle
0,01/01,6am,4-6ft,4.5ft,13s,7,15,mph,19°c,100%,NW,311,Offshore,E,87
1,01/01,12pm,4-6ft,4.5ft,13s,10,13,mph,23°c,100%,NW,312,Cross/Offshore,E,85
2,01/01,6pm,4-7ft,4.5ft,14s,8,14,mph,20°c,100%,NW,313,Cross/Offshore,ENE,68
3,02/01,6am,5-8ft,5.5ft,14s,8,16,mph,19°c,100%,NW,317,Cross/Offshore,E,85
4,02/01,12pm,7-10ft,7ft,16s,11,15,mph,22°c,100%,NW,319,Cross/Offshore,ENE,70


In [213]:
df_pipe_2018.to_csv("df_pipe_2018.csv", index=False)

###  Mundaka

In [224]:
df_mundaka_2018.head()

,date,time,range,height,period,wind_mph,wind_kmh,mph,temp,prob,swell_dir,swell_angle,wind,wind_dir,wind_angle
0,01/01,6am,8-12ft,16ft,15s,21,29,mph,11°c,100%,NW,304,Offshore,SSW,204
1,01/01,12pm,7-12ft,14.5ft,15s,25,40,mph,15°c,100%,WNW,303,Offshore,SW,224
2,01/01,6pm,10-16ft,22ft,15s,31,47,mph,14°c,100%,NW,306,Cross-shore,WNW,297
3,02/01,6am,7-11ft,13.5ft,15s,17,27,mph,13°c,100%,NW,307,Offshore,SSW,214
4,02/01,12pm,6-9ft,11ft,14s,15,24,mph,14°c,100%,NW,304,Offshore,SW,223


In [225]:
df_mundaka_2018.to_csv("df_mundaka_2018.csv", index=False)

### Fronton

In [236]:
df_fronton_2018.head()

,date,time,range,height,period,wind_mph,wind_kmh,mph,temp,prob,swell_dir,swell_angle,wind,wind_dir,wind_angle
0,01/01,6am,2-3ft,3.5ft,6s,12,19,mph,15°c,100%,NNE,29,Onshore,NE,42
1,01/01,12pm,3-5ft,3ft,16s,13,19,mph,18°c,100%,NW,323,Onshore,NE,45
2,01/01,6pm,3-5ft,3.5ft,16s,14,21,mph,17°c,100%,NW,326,Onshore,NNE,31
3,02/01,6am,3-5ft,3.5ft,14s,11,16,mph,15°c,100%,NW,325,Onshore,NE,44
4,02/01,12pm,3-4ft,3ft,14s,13,19,mph,20°c,100%,NW,326,Onshore,ENE,60


In [237]:
df_fronton_2018.to_csv("df_fronton_2018.csv", index=False)

### Box

In [251]:
df_box_2018.head()

,date,time,range,height,period,wind_mph,wind_kmh,mph,temp,prob,swell_dir,swell_angle,wind,wind_dir,wind_angle
0,01/01,6am,6-9ft,9ft,15s,9,12,mph,19°c,100%,SW,224,Offshore,SE,136
1,01/01,12pm,6-9ft,8.5ft,14s,16,17,mph,24°c,100%,SW,227,Cross-shore,S,181
2,01/01,6pm,5-8ft,8ft,14s,16,26,mph,19°c,100%,SW,226,Offshore,SSE,152
3,02/01,6am,5-7ft,6.5ft,13s,7,10,mph,19°c,100%,SW,229,Offshore,ESE,120
4,02/01,12pm,4-7ft,6ft,12s,14,16,mph,26°c,100%,SW,230,Cross/Onshore,SSW,197


In [252]:
df_box_2018.to_csv("df_box_2018.csv", index=False)

###  Puerto escondido


In [256]:
df_pe_2018.head()

,date,time,range,height,period,wind_mph,wind_kmh,mph,temp,prob,swell_dir,swell_angle,wind,wind_dir,wind_angle
0,01/01,6am,3-5ft,1.9ft,16s,5,5,mph,21°c,100%,SSW,214,Offshore,NNE,16
1,01/01,12pm,3-5ft,1.7ft,16s,5,6,mph,31°c,100%,SSW,212,Onshore,SSW,194
2,01/01,6pm,2-4ft,1.2ft,17s,4,5,mph,23°c,100%,SSW,211,Onshore,SW,216
3,02/01,6am,3-5ft,1.8ft,15s,7,10,mph,20°c,100%,SW,214,Offshore,NNE,15
4,02/01,12pm,3-5ft,1.9ft,15s,1,7,mph,34°c,100%,SSW,213,Onshore,S,190


In [257]:
df_pe_2018.to_csv("df_pe_2018.csv", index=False)

### Itacoatiara

In [261]:
df_ita_2018.head()

,date,time,range,height,period,wind_mph,wind_kmh,mph,temp,prob,swell_dir,swell_angle,wind,wind_dir,wind_angle
0,01/01,6am,1ft,0.8ft,10s,5,6,mph,25°c,100%,SSW,192,Cross/Offshore,W,281
1,01/01,12pm,1ft,0.8ft,10s,12,12,mph,28°c,100%,SSW,194,Onshore,S,173
2,01/01,6pm,1ft,0.8ft,9s,9,11,mph,26°c,100%,SSW,194,Cross/Onshore,ESE,104
3,02/01,6am,1-2ft,1.8ft,8s,4,4,mph,25°c,100%,S,181,Offshore,NNE,27
4,02/01,12pm,2ft,1.9ft,8s,9,10,mph,30°c,100%,S,177,Onshore,SE,132


In [262]:
df_ita_2018.to_csv("df_ita_2018.csv", index=False)

### Padang Padang

In [293]:
df_pp_2018.head()

,date,time,range,height,period,wind_mph,wind_kmh,mph,temp,prob,swell_dir,swell_angle,wind,wind_dir,wind_angle
0,01/01,6am,Plano,6ft,13s,17,19,mph,27°c,100%,SSW,206,Onshore,W,280
1,01/01,12pm,Plano,5.5ft,12s,11,12,mph,27°c,100%,SSW,204,Onshore,W,263
2,01/01,6pm,Plano,5.5ft,12s,6,6,mph,28°c,100%,SSW,202,Onshore,SW,220
3,02/01,6am,Plano,5ft,12s,4,3,mph,27°c,100%,SSW,202,Offshore,NNE,28
4,02/01,12pm,Plano,4.5ft,12s,7,7,mph,28°c,100%,SSW,201,Cross/Onshore,SE,143


In [294]:
df_pp_2018.to_csv("df_pp_2018.csv", index=False)

### Mosca

In [298]:
df_mosca_2018.head()

,date,time,range,height,period,wind_mph,wind_kmh,mph,temp,prob,swell_dir,swell_angle,wind,wind_dir,wind_angle
0,01/01,6am,2-3ft,3.5ft,6s,12,19,mph,15°c,100%,NNE,29,Onshore,NE,42
1,01/01,12pm,2-3ft,4ft,6s,13,19,mph,18°c,100%,NNE,32,Onshore,NE,45
2,01/01,6pm,2-3ft,3.5ft,7s,14,21,mph,17°c,100%,NNE,31,Onshore,NNE,31
3,02/01,6am,2-3ft,4.5ft,7s,11,16,mph,15°c,100%,NNE,33,Onshore,NE,44
4,02/01,12pm,2-3ft,4.5ft,7s,13,19,mph,20°c,100%,NE,35,Onshore,ENE,60


In [299]:
df_mosca_2018.to_csv("df_mosca_2018.csv", index=False)